In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import binom
np.random.seed(40)
n = 100 #total cptys
S = 125 #no. of days
m = 80 #N(0,1) cptys
d = 0.2 #additional deviation
N = 1000 #no. of simulations

In [3]:
#Return data for 1st m counterparties
returns1 = np.random.normal(loc = 0, scale = 1, size = (m, S, N))
column_names1 = [f'Stock_{i}_simu{j}' for i in range (1,m+1) for j in range(1,N+1)]
stock_data1 = pd.DataFrame(data = returns1.reshape(S,m*N),columns = column_names1)

#Return data for lst n-m counterparties
returns2 = np.random.normal(loc = 0, scale = 1+d, size = (n-m, S, N))
column_names2 = [f'Stock_{i}_simu{j}' for i in range (m+1,n+1) for j in range(1,N+1)]
stock_data2 = pd.DataFrame(data = returns2.reshape(S,(n-m)*N),columns = column_names2)

#Combine the two data
stock_data = pd.concat([stock_data1,stock_data2], axis = 1, join = 'outer')
stock_data.head()

,Stock_1_simu1,Stock_1_simu2,Stock_1_simu3,Stock_1_simu4,Stock_1_simu5,Stock_1_simu6,Stock_1_simu7,Stock_1_simu8,Stock_1_simu9,Stock_1_simu10,...,Stock_100_simu991,Stock_100_simu992,Stock_100_simu993,Stock_100_simu994,Stock_100_simu995,Stock_100_simu996,Stock_100_simu997,Stock_100_simu998,Stock_100_simu999,Stock_100_simu1000
0,-0.607548,-0.126136,-0.684606,0.928715,-1.844401,-0.467002,2.292490,0.488810,0.710267,1.055534,...,-1.555039,0.543289,-1.050471,-0.600672,-0.893677,2.545587,-0.098404,0.969962,2.091743,-0.376333
1,1.035659,-1.482473,2.081862,-1.022411,1.223807,-0.868106,-0.458424,-0.246780,-1.044995,0.771701,...,-0.587255,-0.220042,1.351133,1.141058,-1.026269,0.978169,-0.117259,1.710375,1.187126,0.028720
2,-0.257828,0.250256,0.815393,-0.074207,-0.587794,1.628137,-2.348472,-0.494349,0.424833,-1.208073,...,0.327501,1.204549,1.465270,1.468597,-0.360424,-0.396101,0.806156,-0.015994,1.185013,1.255324
3,0.077492,-1.690478,1.308426,-0.442652,-1.052781,-0.831622,-1.346843,-0.163094,0.056581,-0.403300,...,1.009391,0.318962,0.407878,2.178189,-1.558379,0.901034,-0.270617,0.212431,1.928991,-1.858251
4,-1.423194,-1.423931,1.817461,-0.294205,0.753039,-0.697900,-0.756217,0.373620,0.771162,0.811755,...,-0.617550,-0.483505,0.542556,-0.110933,0.050162,1.518952,-1.889847,-1.819877,-0.888752,0.943026


In [4]:
#Qntiles to be checked
qntile = (0.9, 0.95, 0.975, 0.99)

# Counting exceptions for each stock on each simulation out of 125 days
exception_count=pd.DataFrame()
for i in range(len(qntile)):
    exception_count[i] = ((stock_data>norm.ppf(1-(1-qntile[i])/2)) | (stock_data<norm.ppf((1-qntile[i])/2))).sum()
    
exception_count.columns = qntile
exception_count

,0.900,0.950,0.975,0.990
Stock_1_simu1,12,7,4,3
Stock_1_simu2,9,4,1,0
Stock_1_simu3,19,9,3,2
Stock_1_simu4,10,4,2,1
Stock_1_simu5,13,4,2,0
...,...,...,...,...
Stock_100_simu996,25,16,12,8
Stock_100_simu997,27,20,11,6
Stock_100_simu998,27,14,9,2
Stock_100_simu999,21,14,5,0


In [5]:
# Cpty exceedance rate
stock = []
for i in range(1, n+1):
    stock.append(f'stock_{i}')

Stock_index = [element for element in stock for _ in range(N)]
exception_count['Stock'] = Stock_index
exceedance_rate_cpty = exception_count.groupby('Stock', sort = False).sum()/(N*S)
exceedance_rate_cpty

,0.9,0.95,0.975,0.99
Stock,,,,
stock_1,0.099744,0.048536,0.024032,0.009808
stock_2,0.099928,0.049704,0.024800,0.010112
stock_3,0.099192,0.049208,0.024496,0.009888
stock_4,0.100280,0.050448,0.025400,0.010288
stock_5,0.100840,0.050288,0.025472,0.010024
...,...,...,...,...
stock_96,0.169256,0.101688,0.061544,0.031584
stock_97,0.171088,0.102936,0.062328,0.032400
stock_98,0.169672,0.101760,0.061152,0.031568


In [6]:
# qntile exceedance
qntile_exception_rate = pd.DataFrame(exception_count.drop(['Stock'], axis = 1).sum()/(N*S*n))
qntile_exception_rate.columns = ['Exceedance Rate']
qntile_exception_rate

,Exceedance Rate
0.9,0.114250
0.95,0.060528
0.975,0.032381
0.99,0.014394


In [7]:
from scipy.stats import binom
qntile = (0.9, 0.95, 0.975, 0.99)
rate = [1 - x for x in qntile]
crit = pd.DataFrame(binom.ppf(0.95, n=125, p = rate))

In [8]:
# Calculation of K value
dummy_df = exceedance_rate_cpty.apply(lambda x: 1-x)
k_matrix = pd.DataFrame(columns = qntile)
for i in range(len(exceedance_rate_cpty)):
    k_matrix.loc[i] = norm.ppf(qntile)/norm.ppf(dummy_df.iloc[i])
for i in range(len(qntile)):
    k_matrix.iloc[:,i] = k_matrix.iloc[:,i].apply(lambda x: 1 if x<1 else x)

k_matrix.describe()

,0.900,0.950,0.975,0.990
count,100.000000,100.000000,100.000000,100.000000
mean,1.070705,1.060713,1.055796,1.052501
std,0.137964,0.118545,0.108781,0.100837
min,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,1.002580,1.002073,1.002126,1.002569
75%,1.005893,1.005061,1.005282,1.007642
max,1.353525,1.304827,1.280641,1.267570


In [43]:
k_matrix.iloc[80:100].describe()

,0.900,0.950,0.975,0.990
count,20.000000,20.000000,20.000000,20.000000
mean,1.344619,1.297242,1.273359,1.255518
std,0.006161,0.004789,0.004774,0.003719
min,1.334996,1.285815,1.264365,1.248109
25%,1.340968,1.294471,1.269962,1.253640
50%,1.344894,1.297805,1.272784,1.254303
75%,1.348118,1.300339,1.275278,1.257289
max,1.359329,1.308044,1.284266,1.262857


In [9]:
# Exceedance rate simulation wise
Simu = []
for i in range(1, N+1):
    Simu.append(f'Simu_{i}')
Simu_index = Simu*n
exception_count['Simu'] = Simu_index
exceedance_rate_simu = exception_count.groupby('Simu', sort = False).sum()/(n*S)
exceedance_rate_simu

,0.9,0.95,0.975,0.99
Simu,,,,
Simu_1,0.11520,0.05800,0.03048,0.01312
Simu_2,0.11480,0.06008,0.03344,0.01624
Simu_3,0.11552,0.06096,0.03224,0.01408
Simu_4,0.11208,0.06120,0.03304,0.01440
Simu_5,0.12056,0.06456,0.03600,0.01448
...,...,...,...,...
Simu_996,0.11648,0.06016,0.03248,0.01360
Simu_997,0.11440,0.06064,0.03464,0.01584
Simu_998,0.11416,0.06000,0.03200,0.01384


In [10]:
dummy_df = exceedance_rate_simu.apply(lambda x: 1-x)
k_matrix_simu = pd.DataFrame(columns = qntile)
for i in range(len(exceedance_rate_simu)):
    k_matrix_simu.loc[i] = norm.ppf(qntile)/norm.ppf(dummy_df.iloc[i])
for i in range(len(qntile)):
    k_matrix_simu.iloc[:,i] = k_matrix_simu.iloc[:,i].apply(lambda x: 1 if x<1 else x)
exceedance_rate_simu

,0.9,0.95,0.975,0.99
Simu,,,,
Simu_1,0.11520,0.05800,0.03048,0.01312
Simu_2,0.11480,0.06008,0.03344,0.01624
Simu_3,0.11552,0.06096,0.03224,0.01408
Simu_4,0.11208,0.06120,0.03304,0.01440
Simu_5,0.12056,0.06456,0.03600,0.01448
...,...,...,...,...
Simu_996,0.11648,0.06016,0.03248,0.01360
Simu_997,0.11440,0.06064,0.03464,0.01584
Simu_998,0.11416,0.06000,0.03200,0.01384


In [11]:
### Calculation of K for overconfident counterparties

#P-value calculations
P_values = pd.DataFrame(columns = qntile)
for i in range(len(qntile)):
    P_values.iloc[:,i] = exception_count.iloc[:,i].apply(lambda x: 1- binom.cdf(x-1, S, p = 1-qntile[i]))
P_values

,0.900,0.950,0.975,0.990
Stock_1_simu1,0.602918,0.434789,0.381094,0.130684
Stock_1_simu2,0.887846,0.876215,0.957774,1.000000
Stock_1_simu3,0.042782,0.174548,0.607281,0.355813
Stock_1_simu4,0.812589,0.876215,0.822435,0.715292
Stock_1_simu5,0.483984,0.876215,0.822435,1.000000
...,...,...,...,...
Stock_100_simu996,0.000590,0.000528,0.000077,0.000042
Stock_100_simu997,0.000097,0.000004,0.000323,0.001704
Stock_100_simu998,0.000097,0.004026,0.004337,0.355813
Stock_100_simu999,0.012500,0.004026,0.204204,1.000000


In [12]:
#Ranking matrix
ranking_mat = P_values.rank(axis = 1)

#Q matrix
Q_mat = P_values.reset_index(drop = True)/ranking_mat.reset_index(drop = True)*len(qntile)
Q_mat.index = exception_count.index

final_q = Q_mat.min(axis = 1)
OvrConf = final_q < 0.0925
OvrConf.index = exception_count.index
OvrConf

Stock_1_simu1         False
Stock_1_simu2         False
Stock_1_simu3         False
Stock_1_simu4         False
Stock_1_simu5         False
                      ...  
Stock_100_simu996      True
Stock_100_simu997      True
Stock_100_simu998      True
Stock_100_simu999      True
Stock_100_simu1000     True
Length: 100000, dtype: bool

In [13]:
OvrConf_exception = exception_count[OvrConf.values]
exceedance_rate_Overconf =OvrConf_exception.groupby('Simu', sort = False).mean()/S
exceedance_rate_Overconf.sort_values(by = ['Simu'])

,0.9,0.95,0.975,0.99
Simu,,,,
Simu_1,0.168941,0.099294,0.063059,0.033412
Simu_10,0.181895,0.096000,0.057684,0.029895
Simu_100,0.160762,0.102095,0.065524,0.034286
Simu_1000,0.178824,0.099294,0.058824,0.033412
Simu_101,0.180000,0.111000,0.062500,0.033500
...,...,...,...,...
Simu_995,0.172800,0.103200,0.064000,0.030400
Simu_996,0.178667,0.110222,0.071556,0.035556
Simu_997,0.178526,0.106526,0.071158,0.041263


In [15]:
dummy_df = exceedance_rate_Overconf.apply(lambda x: 1-x)
k_matrix_OvrConf = pd.DataFrame(columns = qntile)
for i in range(len(exceedance_rate_Overconf)):
    k_matrix_OvrConf.loc[i] = norm.ppf(qntile)/norm.ppf(dummy_df.iloc[i])
for i in range(len(qntile)):
    k_matrix_OvrConf.iloc[:,i] = k_matrix_simu.iloc[:,i].apply(lambda x: 1 if x<1 else x)
k_matrix_OvrConf.describe()

,0.900,0.950,0.975,0.990
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,1.064248,1.060924,1.061121,1.063753
std,0.012802,0.011731,0.012072,0.014098
min,1.023940,1.027574,1.028935,1.015096
25%,1.055399,1.052448,1.052439,1.055482
50%,1.064161,1.060450,1.060736,1.063033
75%,1.072596,1.069337,1.069569,1.073567
max,1.102988,1.096980,1.100362,1.112267
